In [1]:
from models.Model import Model
from models.GPT2Generator import GPT2Generator
from models.Logistic.BOWLogistic import BOWLogistic
from models.MLP.BOWMLP import BOWMLP
from models.MLP.CBOW_BOWMLP import CBOW_BOWMLP
from models.MLP.CBOWMLP import CBOWMLP
from models.RandomForest.BOWRandomForest import BOWRandomForest
from models.RandomForest.CBOW_BOWRandomForest import CBOW_BOWRandomForest
from models.RandomForest.CBOWRandomForest import CBOWRandomForest
from models.Logistic.CBOW_BOWLogistic import CBOW_BOWLogistic
from models.Logistic.CBOWLogistic import CBOWLogistic
from models.XGBoost.BOWXGBoost import BOWXGBoost
from models.XGBoost.CBOW_BOWXGBoost import CBOW_BOWXGBoost
from models.XGBoost.CBOWXGBoost import CBOWXGBoost

from models.SequenceLabeller_BiLSTM_CRF import SequenceLabeller_BiLSTM_CRF
from models.SequenceLabeller_BiLSTM_CRF_Beam import SequenceLabeller_BiLSTM_CRF_Beam
from models.SequenceLabeller_BERT import SequenceLabeller_BERT


from languages.LanguageModel import LanguageModel
from DataExploration import DataExploration
# from languages.Japanese import Japanese
from languages.English import English
from languages.Finnish import Finnish
from Preprocess import Preprocess
from Pipeline import Pipeline
from typing import List
import torch
import datasets
import pandas as pd
import random
import numpy as np

In [2]:
# Download word_vectors
!mkdir word_vectors
%cd word_vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fi.300.vec.gz

!gzip -d cc.en.300.vec.gz
!gzip -d cc.ja.300.vec.gz
!gzip -d cc.fi.300.vec.gz

%cd ..

c:\Users\chris\OneDrive\Dokumenter\GitHub\NLP-2022\word_vectors


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.


c:\Users\chris\OneDrive\Dokumenter\GitHub\NLP-2022


'gzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

enforce_reproducibility()

In [5]:
# Is used to minimize the clutter in the console
datasets.logging.set_verbosity_error()

# Define the languages to be used
languages: List[LanguageModel] = [
    English(),
    Finnish(),
    #Japanese()
]

# gpt2Generator = GPT2Generator()
torch.cuda.empty_cache()


In [6]:
def print_headline(language: str):
    print(f'\n\n--- Language: {language} ---')

# Preprocessing and Dataset Analysis

In [7]:
all_data = {}

for language in languages:
    pipeline = Pipeline()

    # Get the preprocessed data and split it into training and validation data
    preprocessor = Preprocess(language.tokenize, language.clean)
    data = pipeline.get_data(language=language.name, preproccesor=preprocessor)
    train_data, validation_data = pipeline.split_data(data)

    all_data[language.name] = {
        "train_data": train_data,
        "validation_data": validation_data
    }


Loading data...

Loading data...


In [8]:
# Explore the data for each language
for language in languages:
    print_headline(language.name)
    data_exploration = DataExploration(all_data[language.name]["train_data"])
    data_exploration.find_frequent_words()



--- Language: english ---

Most frequent first words:
[('When', 1999), ('What', 1977), ('How', 1182), ('Who', 930), ('Where', 416)]
Most frequent last words:
[('?', 6693), ('zombie', 2), ('metabolite', 2), ('BCE', 2), ('\\', 1)]



--- Language: finnish ---

Most frequent first words:
[('Milloin', 3234), ('Mikä', 2059), ('Missä', 1474), ('Kuka', 1418), ('Mitä', 954)]
Most frequent last words:
[('?', 12290), ('tulitaistelussa', 2), ('syntynyt', 2), ('pinta-ala', 2), ('lintulaji', 2)]



# Binary Question Classification
Binary classfiers that only takes features based on the question, context document and combinations of the two

In [9]:
for language in languages:
    print_headline(language.name)

    feature_based_classifiers = [
        BOWRandomForest(language.name),
        BOWMLP(language.name),
        BOWLogistic(language.name),
        BOWXGBoost(language.name)
    ]
    
    for classifier in feature_based_classifiers:
        print(f'--- Classifier: {classifier.__class__.__name__} ---')
        train_data = all_data[language.name]["train_data"]
        validation_data = all_data[language.name]["validation_data"]

        pipeline = Pipeline()
        pipeline.train(
            model=classifier,
            X=classifier.extract_X(train_data),
            y=classifier.extract_y(train_data)
        )

        pipeline.evaluate(
            model=classifier,
            X=classifier.extract_X(validation_data),
            y=classifier.extract_y(validation_data)
        )
    



--- Language: english ---


TypeError: RandomForest.__init__() takes 1 positional argument but 2 were given

# Representation Learning
Extension of our binary question classifers to also include features based on continous vector representations

In [ ]:
for language in languages:
    print_headline(language.name)

    continous_based_classifiers = [
        CBOW_BOWRandomForest(language.name),
        CBOW_BOWMLP(language.name),
        CBOW_BOWLogistic(language.name),
        CBOW_BOWXGBoost(language.name)
    ]
    
    for classifier in feature_based_classifiers:
        print(f'--- Classifier: {classifier.__class__.__name__} ---')
        train_data = all_data[language.name]["train_data"]
        validation_data = all_data[language.name]["validation_data"]

        pipeline = Pipeline()
        pipeline.train(
            model=classifier,
            X=classifier.extract_X(train_data),
            y=classifier.extract_y(train_data)
        )

        pipeline.evaluate(
            model=classifier,
            X=classifier.extract_X(validation_data),
            y=classifier.extract_y(validation_data)
        )

We also test how the performance if only the continous representations was to be used

In [ ]:
for language in languages:
    print_headline(language.name)

    continous_based_classifiers = [
        CBOWRandomForest(language.name),
        CBOWMLP(language.name),
        CBOWLogistic(language.name),
        CBOWXGBoost(language.name)
    ]
    
    for classifier in feature_based_classifiers:
        print(f'--- Classifier: {classifier.__class__.__name__} ---')
        train_data = all_data[language.name]["train_data"]
        validation_data = all_data[language.name]["validation_data"]

        pipeline = Pipeline()
        pipeline.train(
            model=classifier,
            X=classifier.extract_X(train_data),
            y=classifier.extract_y(train_data)
        )

        pipeline.evaluate(
            model=classifier,
            X=classifier.extract_X(validation_data),
            y=classifier.extract_y(validation_data)
        )

# Language modelling
Extension to the classifiers in which word/sentence representations are instead extracted from neural language models

In [ ]:
for language in languages:
    print_headline(language.name)

    classifier = GPT2CBOWLogistic(language.name)

    train_data = all_data[language.name]["train_data"]
    validation_data = all_data[language.name]["validation_data"]

    pipeline = Pipeline()
    pipeline.train(
        model=classifier,
        X=classifier.extract_X(train_data),
        y=classifier.extract_y(train_data)
    )

    pipeline.evaluate(
        model=classifier,
        X=classifier.extract_X(validation_data),
        y=classifier.extract_y(validation_data)
    )

We try to sample from these language models to see what kinds of sentences they generate. Moreover we measure the performance on the TyDi QA validation data with a commonly used language model evaluations metric

In [ ]:
question_beginning = {
    'english': ['When', 'What', 'How'],
    'finnish': ['Milloin', 'Mikä', 'Missä'],
    'japanese': ['日本', '『', 'アメリカ']
}

for language in languages:
    print_headline(language.name)

    model = GPT2Generator(language.name)

    train_data = all_data[language.name]["train_data"]
    validation_data = all_data[language.name]["validation_data"]

    pipeline = Pipeline()
    pipeline.train(
        model=classifier,
        X=classifier.extract_X(train_data),
        y=classifier.extract_y(train_data)
    )

    for starting_word in question_beginning[language.name]:
        model.generate_text(f'Question: {starting_word}')
    model.get_perplexity(model.extract_X(validation_data))

# Error Analysis and Interpretability

First we load the trained models and then we run the explainability method for that nodel

In [ ]:
models :list[Model] =[
  CBOW_BOWLogistic(),
  CBOW_BOWRandomForest()
]
for model in models:
  model.set_language(language.name)
  model.load()
  model.explainability(n=20)

## Adverserial

In [ ]:
language = English()
pipeline = Pipeline()


preprocessor = Preprocess(language.tokenize, language.clean)
preprocessor.from_datasets = False
raw_adversarial_data = {
  "question_text"      : ["When was Queen Elizabeth II born?" , "What is the name for a male horse?" , "What is the name for a male horse?"],
  "document_plaintext" : [
    "Queen Elizabeth II was the queen of England from 1952 until she died on the 8th of september 2022." ,
    "A horse is frequently referred to as a stallion once he fathers a foal. While in most of the western world stallions are primarily kept for breeding, it is popular in parts of the Middle East and Asia for stallions to be used for riding (almost always by men)." ,
    "A mare is an adult female horse or other equine. In most cases, a mare is a female horse over the age of three, and a filly is a female horse three and younger. In Thoroughbred horse racing, a mare is defined as a female horse more than four years old. The word can also be used for other female equine animals, particularly mules and zebras, but a female donkey is usually called a 'jenny'. A broodmare is a mare used for breeding. A horse's female parent is known as its dam."]
}
dataset = pd.DataFrame.from_dict(data = raw_adversarial_data)
data = preprocessor.preprocess(dataset)


models :list[Model] =[
  CBOW_BOWLogistic(),
  CBOW_BOWRandomForest()
]
for model in models:
  model.set_language(language.name)
  model.load()

  X_validation = model.extract_X(data)
  y_validation = [0,1,0]

  pipeline.evaluate(
              model,
              X_validation,
              y_validation
          )

  print(model.predict(X_validation))

# Sequence Labelling
We implement a sequence labeller, which predicts which parts of a paragraph are likel part of the answer to the corresponding question

In [ ]:
config = {
    'num_train_epochs': 10,
    'learning_rate': 2e-5,
    'per_device_train_batch_size': 8,
    'per_device_eval_batch_size': 8,
    'warmup_steps': 200,
    'weight_decay': 0.01,
    'lstm_dim': 128,
    'dropout_prob': 0.1,
    'n_workers': 0,
    'beam_size': 2,
}

In [ ]:
import os
os.environ["WANDDB_DISABLED"] = "true"

In [ ]:
for language in languages:
    print_headline(language.name)

    sequence_labellers = [
        SequenceLabeller_BERT(language.name, config),
        SequenceLabeller_BiLSTM_CRF(language.name, config),
    ]

    for sequence_labeller in sequence_labellers:
        print(f'--- Sequence Labeller: {sequence_labeller.__class__.__name__} ---')
        train_data = all_data[language.name]["train_data"]
        validation_data = all_data[language.name]["validation_data"]

        pipeline = Pipeline()
        pipeline.train(
            model=sequence_labeller,
            X=sequence_labeller.extract_X(train_data),
            y=sequence_labeller.extract_y(train_data)
        )

        pipeline.evaluate(
            model=sequence_labeller,
            X=sequence_labeller.extract_X(validation_data),
            y=sequence_labeller.extract_y(validation_data)
        )
        sequence_labeller.save()



--- Language: english ---


loading configuration file config.json from cache at C:\Users\chris/.cache\huggingface\hub\models--bert-base-uncased\snapshots\5546055f03398095e385d7dc625e636cc8910bf2\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\chris/.cache\huggingface\hub\models--bert-base-uncased\snapshots\5546055f03398095e385d7dc625e636cc8910bf2\vocab.t

--- Sequence Labeller: SequenceLabeller_BERT ---


  0%|          | 0/7 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokenized_question, tokenized_plaintext, __index_level_0__, tags. If tokenized_question, tokenized_plaintext, __index_level_0__, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
c:\Users\chris\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warning

Training the model...


AttributeError: module 'wandb' has no attribute 'run'

We add an extension to the sequence labeller which uses beam search to select the optimal sequence of labels for the location of the answer in the text. Analyse how the performance of this system differs with beam search.

In [ ]:
for language in languages:
    print_headline(language.name)

    sequence_labeller = SequenceLabeller_BiLSTM_CRF_Beam(language.name, config)

    train_data = all_data[language.name]["train_data"]
    validation_data = all_data[language.name]["validation_data"]

    pipeline = Pipeline()
    pipeline.train(
        model=sequence_labeller,
        X=sequence_labeller.extract_X(train_data),
        y=sequence_labeller.extract_y(train_data)
    )

    num_beams = [1, 2, 3]

    for beam in num_beams:
        sequence_labeller.beam_size = beam
        pipeline.evaluate(
            model=sequence_labeller,
            X=sequence_labeller.extract_X(validation_data),
            y=sequence_labeller.extract_y(validation_data)
        )
    
    sequence_labeller.save()



--- Language: english ---


/Users/christianjensen/Library/CloudStorage/OneDrive-UniversityofCopenhagen/2 - Projects/NLP Course/QuestionAnswering/models/SequenceLabeller_BiLSTM_CRF_Beam.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset.apply(lambda row: np.concatenate((['<START>'], row['tokenized_question'], ['<SEP>'], row['tokenized_plaintext'], ['<STOP>'])), axis=1)
/Users/christianjensen/Library/CloudStorage/OneDrive-UniversityofCopenhagen/2 - Projects/NLP Course/QuestionAnswering/models/SequenceLabeller_BiLSTM_CRF_Beam.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Training the model...


  0%|          | 0/2 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

tensor([[    4,  1450,    16,     8,  4222,    12,  1058, 16188,    48,     4,
         30016,     7, 30038,    34,    52,   139,    29,   217,    12,    61,
            12,     8,  7332,   435,   406,   528,  1153,     9,    28,  1396,
            12, 30045,    15,  9239,  4370,    11,     8, 30040, 30009,    15,
             8, 30042,  1194,    36, 11112,     8,   339,    12, 30017,  1921,
           527,    11, 17729,   871,    11, 10273,     8,  2631,    12, 15933,
             7,   118,    26, 14200,     7,   118,    26,  9253,    15,     8,
           966,     7,    10,    82,   492,   118,  3630,   292,    35, 15266,
             9,    67,  3603,   436,   254,  5398,    11,   191,     8,  1388,
             7,    47,    25,  9239,  1114,     8,   143, 15933,    29,  1809,
          8376,    30,    13,   614,  6142,   546,  1270,     9,   169,   367,
           167,   169,   324,   167,     4]])
tensor([[    4,   236,    29,     8,   107,  2772,   706,    48,     4, 10419,
      

/opt/homebrew/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

In [ ]:
# Qualitative investigation of the predicted answer spans

# Multilingual QA

We start by implementing our binary question system with a multilingual encoder instead of the monolingual ones. With this we perform zero-shot cross-lingual evaluation

In [ ]:
# Mutli-lingual binary classification

We now implement our sequence tagger with a multilingual encoding and perform zero-shot cross-lingual evaluation

In [ ]:
# Zero shot classification
for language in languages:
    print(f'\n\n--- Training on {language.name} ---')

    sequence_labeller = SequenceLabeller_BERT('multilingual', config)

    train_data = all_data[language.name]["train_data"]
    validation_data = all_data[language.name]["validation_data"]


    pipeline = Pipeline()
    pipeline.train(
        model=sequence_labeller,
        X=sequence_labeller.extract_X(train_data),
        y=sequence_labeller.extract_y(train_data)
    )

    for language in languages:
      print(f'--- Validating on {language.name} ---')
      validation_data = all_data[language.name]['validation_data']
      pipeline.evaluate(
          model=sequence_labeller,
          X=sequence_labeller.extract_X(validation_data),
          y=sequence_labeller.extract_y(validation_data)
      )
    
    sequence_labeller.save()



--- Training on english ---
--- Validating on finnish ---


loading configuration file config.json from cache at /Users/christianjensen/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": 

  0%|          | 0/7 [00:00<?, ?ba/s]

KeyboardInterrupt: 